In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/study/recycle/object_detection

import cv2
import numpy as np

LABELS = ['recycle','trash']  # 나중에 여러분들 라벨을 여기에 적어주시면 됩니다.

CONFIDENCE = 0.3
THRESHOLD = 0.3  #NMS(Num Max Suppression)

# 고라니와 너구리 사진을 학습시킨 가중치와 환경 구성 파일을 불러와서 모델을 생성
net = cv2.dnn.readNetFromDarknet('yolov4_custom.cfg','yolov4_custom_final.weights')

classes =['recycle','trash']  # 나중에 여러분들 라벨을 여기에 적어주시면 됩니다.

cap = cv2.VideoCapture('recycle_test1.mp4')  # 원본 동영상 이름을 여러분들 걸로 적어주시면 됩니다.

font = cv2.FONT_HERSHEY_PLAIN  # 동영상에 표시할 글씨 폰트 지정
colors = np.random.uniform(0, 255, size=(100, 3))  # 네모 박스 색깔과 글씨 색깔이 랜덤으로 결정되게 함

# 동영상 가로, 세로 프레임 사이즈
frame_size = (cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 디텍션한 동영상의 확장자를 mp4 로 하겠다.
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # <-------------------  이 부분 

# 사물 검출된 동영상이 detections.mp4 로 저장됩니다.
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter('detections1.mp4', fourcc, fps, (int(frame_size[0]), int(frame_size[1])))

while True:  # 무한 루프문을 수행합니다.
    ret, img = cap.read()  # 원본 동영상 이미지를 한 프레임씩 가져옵니다.
    if not ret:            # 더 이상 가져올 프레임이 없다면
        break              # 무한 루프를 종료하겠다

    # 동영상의 넓이와 높이 사이즈 정보를 가져옵니다.
    width,height = (cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # 동영상의 한 프레임의 사진을 정규화하고 사이즈를 416×416 으로 변경합니다.
    blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False)
    # crop=False 는 주변의 가장자리를 잘라내는 것입니다.
    # 신경망을 학습할 때도 정규화를 했기 때문에 여기서도 정규화 (1/255)
    # swapRB=True 는 opencv 가 RGB 가 아니라 BGR 로 이미지 처리를해서 입니다.
    # RGB -> BGR 로 변경하려면 True 로 해야합니다.

    # 학습된 가중치와 환경 구성 파일이 셋팅된 다크넷 신경망에 전처리된 frame 을 넣습니다.
    net.setInput(blob)

    # 다크넷 신경망에서 어느정도 결론이 난 맨 마지막 층들의 이름을 가져와서 변수에 담습니다.
    output_layers_names = net.getUnconnectedOutLayersNames()

    # 마지막 층들을 신경망에 흘려보냅니다.
    layerOutputs = net.forward(output_layers_names)
    # 하나의 사진(프레임)당 박싱을 하는 윈도우들이 보통 507개가 생기는데
    # 그 507개에 해당하는 frame 데이터가 layerOutputs 에 저장됩니다.

    # 사물 검출을 위해서 사용할 비어있는 리스트 3개를 선언합니다.
    boxes = []
    confidences = []
    class_ids = []

    for output in layerOutputs:  # 507개의 박스 데이터를 하나씩 불러옵니다.
        for detection in output:  # 리스트 안에 리스트로 되어있어서 한 번 더 for 문을 사용해서
                                  # 리스트 안에 요소를 가져옵니다.
            scores = detection[5:]  # 507개의 박스에서 고라니와 너구리일 확률이 scores 에 담깁니다.
            class_id = np.argmax(scores)  # [0.960522, 0] <- 이렇게 되면 0이 class_id 에 담깁니다.
            confidence = scores[class_id]  # [0.960522, 0][0] 이니까 0.960522 가 confidence 에 담깁니다.
            if confidence > 0.2:  # 확률이 0.2 보다 크다면
                center_x = int(detection[0]*width)  # 0~1 사이로 변경된 좌표값을 실제 좌표값으로 변경
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)  # 실제 가로
                h = int(detection[3]*height)  # 실제 높이로 변경합니다.

                x = int(center_x - w/2)  # 박스의 중심 x좌표
                y = int(center_y - h/2)  # 박스의 중심 y좌표

                boxes.append([x, y, w, h])  # 원하는 사물이 있는 박스 정보를 boxes 에 넣습니다.
                confidences.append((float(confidence)))  # 0.2 보다 큰 확률 데이터를 confidence 에 입력
                class_ids.append(class_id)  # 0 아니면 1이 class_ids 에 담깁니다.

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.2, 0.4)
    # 여기서 0.2 는 사물이 맞을 확률에 대한 임계치이고 0.4 는 박스 안에 사물이 있을 확률에 대한
    # 임계치를 지정해서 이 임계치보다 더 큰 박스 정보들만 indexes 에 담습니다.

    if len(indexes) > 0:  # indexes 에 값이 있다면
        for i in indexes.flatten():  # boxes 리스트(x,y,w,h)에 대한 정보를 flatten 시켜서
            x, y, w, h = boxes[i]  # x, y, w, h 변수에 넣습니다.
            label = str(classes[class_ids[i]])  # 고라니인지 너구리인지를 label 에 넣고
            confidence = str(round(confidences[i],2))  # 고라니일 확률 또는 너구리일 확률을
                                                       # confidence 에 넣습니다.
            color = colors[i]  # 랜덤으로 정한 색깔을 color 에 넣고
            cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)  # 네모 박스를 그립니다.
            cv2.putText(img, label + " " + confidence, (x, y+20), font, 2, (255,255,255), 2)

    out.write(img)  # 동영상을 기록합니다.
    
    key = cv2.waitKey(1)  # 중간에 취소를 하고 싶어서 키보드의 key 정보를 key 에 넣고
    if key==27:  # key 가 27번은 취소키(esc)를 눌렀다는 것입니다.
        break    # 키보드에서 취소키를 눌렀다면 멈춥니다.

out.release()  # 디텍션된 동영상에 대한 데이터가 있는 변수를 지웁니다.
cap.release()  # 원본 동영상

/content/drive/MyDrive/study/recycle/object_detection
